In [ ]:
import sys
import importlib
import numpy as np
from skimage import filters
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm import tqdm
from tqdm import trange

sys.path.append('../../')
from tools import plotting as myplt
from tools import beam_analysis as ba
from tools import ap_utils
from tools import utils
from measurement.tomography import reconstruct as rec

In [ ]:
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['axes.grid'] = False
pplt.rc['grid.alpha'] = 0.04
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'mono_r'

Load distribution from Holmes (2018).

In [ ]:
X = np.loadtxt('/Users/austin/Dropbox/SCBD/scratch/RUN_0167/data/Bm_Parts_0_300')
X = X[:, :4]
X -= np.mean(X, axis=0)

Normalize the distribution. 

In [ ]:
Sigma = np.cov(X.T)
alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
print(alpha_x, alpha_y, beta_x, beta_y)

np.random.seed(1)
dalpha = 0.0
dbeta = 0.0 * max(beta_x, beta_y)
alpha_x += np.random.uniform(-dalpha, dalpha)
alpha_y += np.random.uniform(-dalpha, dalpha)
beta_x += np.random.uniform(-dbeta, dbeta)
beta_y += np.random.uniform(-dbeta, dbeta)
print(alpha_x, alpha_y, beta_x, beta_y)

In [ ]:
V = ap_utils.V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
Vinv = np.linalg.inv(V)
Xn = utils.apply(Vinv, X)

Bin the coordinates at the reconstruction location.

In [ ]:
n_bins = 75
cmap = 'mono_r'

In [ ]:
axes = myplt.corner(Xn, figsize=(6, 6), cmap=cmap, bins=n_bins, 
                    autolim_kws=dict(sigma=3.65)
                   )
rec_limits = [ax.get_xlim() for ax in axes[-1, :]]

In [ ]:
Z_true, rec_edges = np.histogramdd(Xn, n_bins, rec_limits, density=True)
rec_centers = [rec.get_centers(_edges) for _edges in rec_edges]
bin_volume = rec.get_bin_volume(rec_limits, n_bins)

Transport the distribution to the screen. Assume the phase advances are evenly spaced over 180 degrees. The Twiss parameters at the screen are randomly chosen.

In [ ]:
K = 15 # number of horizontal optics settings
L = 15 # number of vertical optics settings

phase_adv_x = np.linspace(0.0, np.pi, K, endpoint=False)
betas_x = np.random.uniform(55.0, 70.0, size=K)
alphas_x = np.random.uniform(0.0, 1.0, size=K)

phase_adv_y = np.linspace(0.0, np.pi, L, endpoint=False)
betas_y = np.random.uniform(5.5, 6.5, size=L)
alphas_y = np.random.uniform(-1.0, 0.0, size=L)

xx_list, tmats_x = [], []
for k in trange(K):
    P = utils.rotation_matrix(phase_adv_x[k])
    V_screen = ap_utils.V_matrix_2x2(alphas_x[k], betas_x[k])
    M = np.linalg.multi_dot([V_screen, P, Vinv[:2, :2]])
    xx_list.append(utils.apply(M, X[:, :2])[:, 0])
    tmats_x.append(np.matmul(M, V[:2, :2]))

yy_list, tmats_y = [], []
for l in trange(L):
    P = utils.rotation_matrix(phase_adv_y[l])
    V_screen = ap_utils.V_matrix_2x2(alphas_y[l], betas_y[l])
    M = np.linalg.multi_dot([V_screen, P, Vinv[2:, 2:]])
    yy_list.append(utils.apply(M, X[:, 2:])[:, 0])
    tmats_y.append(np.matmul(M, V[2:, 2:]))

Bin the coordinates on the screen.

In [ ]:
xmax_screen = ymax_screen = 75.0 # [mm]
screen_edges = [
    np.linspace(-xmax_screen, xmax_screen, n_bins + 1),
    np.linspace(-xmax_screen, xmax_screen, n_bins + 1),
]
screen_centers = [rec.get_centers(_edges) for _edges in screen_edges]

In [ ]:
S = np.zeros((n_bins, n_bins, K, L))
for k, xx in enumerate(tqdm(xx_list)):
    for l, yy in enumerate(yy_list):
        S[:, :, k, l], _, _ = np.histogram2d(xx, yy, bins=screen_edges)

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=5, figwidth=8.5, space=0.25)
for k, ax in enumerate(axes):
    ax.pcolormesh(S[:, :, k, 0].T, cmap='mono_r')
    ax.annotate(r'$\mu_x = {:.0f}\degree$'.format(np.degrees(phase_adv_x[k])), 
                xy=(0.01, 0.93), xycoords='axes fraction', color='white', fontsize='small')
axes.format(xlabel=r"$x$", ylabel=r"$y$", xticks=[], yticks=[])
plt.savefig('_output/tomo_sim_target_scan.png', dpi=350)
plt.show()

In [ ]:
fig, axes = pplt.subplots(nrows=K, ncols=L, figwidth=10, space=0.1)
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        ax = axes[i, j]
        ax.pcolormesh(S[:, :, i, j].T, cmap='mono_r', ec='None')
axes.format(
    xticks=[], yticks=[], 
#     xlabel='x', ylabel='y',
)
plt.savefig('_output/tomo_sim_target_scan_full.png', dpi=500)
plt.show()

## Hock's method

In [ ]:
Z_hock = rec.hock4D(S, screen_centers, [rec_centers[0], rec_centers[2]], 
                    tmats_x, tmats_y, method='SART', iterations=3)
Z_hock = rec.process(Z_hock, keep_positive=True, density=True, limits=rec_limits)

In [ ]:
Z_hock = rec.process(Z_hock, density=True, limits=rec_limits)

In [ ]:
print('min(Z) = {}'.format(np.min(Z_hock)))
print('max(Z) = {}'.format(np.max(Z_hock)))
print('sum(Z) * bin_volume = {}'.format(np.sum(Z_hock) * bin_volume))
print()
print('min(Z_true) = {}'.format(np.min(Z_true)))
print('max(Z_true) = {}'.format(np.max(Z_true)))
print('sum(Z_true) * bin_volume = {}'.format(np.sum(Z_true) * bin_volume))

In [ ]:
avg_abs_err_per_bin = np.sum(np.abs(Z_hock - Z_true)) / Z_hock.size
print('Average absolute error per bin = {}'.format(avg_abs_err_per_bin))

In [ ]:
fig, axes = pplt.subplots(ncols=4, figsize=(6, 2), spanx=False)
labels = ["x", "x'", "y", "y'"]
for i in range(4):
    axes[i].plot(rec_centers[i], rec.project(Z_true, i), color='black', label='True')
    axes[i].plot(rec_centers[i], rec.project(Z_hock, i), color='red8', ls='dotted', label='Reconstructed')
    axes[i].set_xlabel(labels[i])
axes[0].legend(loc=(0.0, 1.02), framealpha=0.0, ncol=1);
plt.savefig('_output/tomo_sim_rec_hock_proj_1D.png', dpi=500)
plt.show()

In [ ]:
def compare_hor(Z, Z_true, cmap=None, cmap_div=None, space=0.3):
    plot_kws = dict(cmap=cmap, ec='None')
    plot_kws_div = dict(diverging=True, cmap=cmap_div, ec='None')
    indices = [(0, 1), (2, 3), (0, 2), (0, 3), (2, 1), (1, 3)]
    fig, axes = pplt.subplots(nrows=3, ncols=6, figwidth=8.5, sharex=False, sharey=False, space=0.3)
    for col, (i, j) in enumerate(indices):
        _Z_true = rec.project(Z_true, [i, j])
        _Z = rec.project(Z, [i, j])
        axes[0, col].pcolormesh(rec_centers[i], rec_centers[j], _Z_true.T, **plot_kws)
        axes[1, col].pcolormesh(rec_centers[i], rec_centers[j], _Z.T, **plot_kws)
        axes[2, col].pcolormesh(rec_centers[i], rec_centers[j], (_Z - _Z_true).T, **plot_kws_div)
        axes[0, col].annotate('{}-{}'.format(labels[i], labels[j]),
                              xy=(0.02, 0.92), xycoords='axes fraction', color='white', 
                              fontsize='medium')
    for ax, title in zip(axes[:, 0], ['True', 'Reconstructed', 'Error']):
        ax.set_ylabel(title)
    axes.format(xticks=[], yticks=[])
    return axes

In [ ]:
def compare_ver(Z, Z_true, cmap=None, cmap_div=None):
    space = 0.35
    plot_kws = dict(cmap=cmap, ec='None')
    plot_kws_div = dict(diverging=True, cmap=cmap_div, ec='None',
                        colorbar=True, 
                        colorbar_kw=dict(width=0.075, space=space, ticklabelsize='small'))
    indices = [(0, 1), (2, 3), (0, 2), (0, 3), (2, 1), (1, 3)]
    fig, axes = pplt.subplots(nrows=6, ncols=3, figsize=(5.0, 8.5), 
                              sharex=False, sharey=False, space=space)
    for k, (i, j) in enumerate(indices):
        _Z_true = rec.project(Z_true, [i, j])
        _Z = rec.project(Z, [i, j])
        axes[k, 0].pcolormesh(rec_centers[i], rec_centers[j], _Z_true.T, **plot_kws)
        axes[k, 1].pcolormesh(rec_centers[i], rec_centers[j], _Z.T, **plot_kws)
        axes[k, 2].pcolormesh(rec_centers[i], rec_centers[j], (_Z - _Z_true).T, **plot_kws_div)
        axes[k, 0].annotate('{}-{}'.format(labels[i], labels[j]),
                            xy=(0.02, 0.92), xycoords='axes fraction', color='white', 
                            fontsize='medium')
    for ax, title in zip(axes[0, :], ['True', 'Reconstructed', 'Error']):
        ax.set_title(title)
    axes.format(xticks=[], yticks=[])
    return axes

In [ ]:
axes = compare_ver(Z_hock, Z_true)
plt.savefig('_output/tomo_sim_rec_hock_proj_2D_ver.png', dpi=500)
plt.show()

axes = compare_hor(Z_hock, Z_true)
plt.savefig('_output/tomo_sim_rec_hock_proj_2D_hor.png', dpi=500)
plt.show()

Transform to unnormalized phase space.

In [ ]:
# Zt = rec.transform(Z, V, centers, new_grid=None)

Study 2D reconstruction as function of number of angles and of the angular range.

In [ ]:
# from skimage.transform import radon, iradon_sart

# i1, i2 = 2, 3
# bins = 50
# z_true, _, _ = np.histogram2d(Xn[:, i1], Xn[:, i2], bins, [limits[i1], limits[i2]], density=True)

# n_proj_list = np.arange(3, 30)
# max_angles = np.linspace(5., 180., 50, endpoint=False)
# errors = np.zeros((len(n_proj_list), len(max_angles)))    
# for i in trange(errors.shape[0]):
#     for j in range(errors.shape[1]):
#         angles = np.linspace(0., max_angles[j], n_proj_list[i])
#         projections = radon(z_true, theta=angles, circle=True)
#         z = iradon_sart(projections, theta=angles)
#         z = iradon_sart(projections, theta=angles, image=z)
#         z = rec.process(z, keep_positive=True, density=True, limits=[limits[i1], limits[i2]])
#         # if i == 0:
#         #     fig, axes = pplt.subplots(ncols=2)
#         #     axes[0].pcolormesh(xedges, yedges, Z_true.T, cmap='viridis',)
#         #     axes[1].pcolormesh(xedges, yedges, Z.T, cmap='viridis',)
#         #     plt.show()
#         errors[i, j] = np.sqrt(np.sum((z - z_true)**2))

In [ ]:
# fig, ax = pplt.subplots()
# ax.pcolormesh(
#     n_proj_list, max_angles, errors.T, cmap=pplt.Colormap('mono'), 
#     vmin=0,
#     colorbar=True, 
#     colorbar_kw=dict(label=r'$\Vert Z - Z_{true} \Vert$', width=0.1, space=1.0)
# )
# ax.format(yformatter='deg', ylabel='Angular range', xlabel='Number of projections')
# plt.savefig('_output/tomo_sim_art2d.png', dpi=350)
# plt.show()

## PIC

In [ ]:
importlib.reload(rec)

In [ ]:
projections = []
tmats = []
for k in np.arange(0, 15, 2):
    for l in np.arange(0, 15, 2):
        projections.append(S[:, :, k, l])
        M = np.zeros((4, 4))
        M[:2, :2] = tmats_x[k]
        M[2:, 2:] = tmats_y[l]
        tmats.append(M)
Z_pic, _proj = rec.pic4D(projections, tmats, rec_centers, screen_edges, max_iters=15)

In [ ]:
compare(Z_pic, Z_true)

## 4D ART 

In [ ]:
n_bins = 50
Z_true2, edges = np.histogramdd(Xn, n_bins, limits, density=True)
centers = []
for _edges in edges:
    centers.append(0.5 * (_edges[:-1] + _edges[1:]))
rec_grid_centers = centers
screen_edges_x = edges[0]
screen_edges_y = edges[2]
bin_volume = rec.get_bin_volume(limits, n_bins)

In [ ]:
K = 8 # number of angles in x dimension
L = 8 # number of angles in y dimension
muxx = np.linspace(0., np.pi, K, endpoint=False)
muyy = np.linspace(0., np.pi, L, endpoint=False)

xx_list, tmats_x = [], []
for mux in tqdm(muxx):
    Mx = utils.rotation_matrix(mux)
    xx_list.append(utils.apply(Mx, Xn[:, :2])[:, 0])
    tmats_x.append(Mx)

yy_list, tmats_y = [], []
for muy in tqdm(muyy):
    My = utils.rotation_matrix(muy)
    yy_list.append(utils.apply(My, Xn[:, 2:])[:, 0])
    tmats_y.append(My)

In [ ]:
sigma = 0.0 # Gaussian blur
projections, tmats = [], []
for k, xx in enumerate(tqdm(xx_list)):
    for l, yy in enumerate(yy_list):
        z, _, _ = np.histogram2d(xx, yy, n_bins, (limits[0], limits[2]))
        z = filters.gaussian(z, sigma)
        projections.append(z)
        M = np.zeros((4, 4))
        M[:2, :2] = tmats_x[k]
        M[2:, 2:] = tmats_y[l]
        tmats.append(M)

In [ ]:
Z = rec.art4D(projections, tmats, rec_grid_centers, (screen_edges_x, screen_edges_y))

In [ ]:
Z = rec.process(Z, keep_positive=True, density=True, limits=limits)

In [ ]:
fig, axes = pplt.subplots(ncols=4, figsize=(6, 2), spanx=False)
labels = ["x", "x'", "y", "y'"]
for i in range(4):
    axes[i].plot(centers[i], rec.project(Z_true2, i), color='black', label='True')
    axes[i].plot(centers[i], rec.project(Z, i), color='red8', ls='dotted', label='Reconstructed')
    axes[i].set_xlabel(labels[i])
axes[0].legend(loc=(0.0, 1.02), framealpha=0.0, ncol=1);
plt.savefig('_output/tomo_sim_rec_art_proj1D.png', dpi=350)
plt.show()

In [ ]:
plot_kws = dict(cmap=cmap)
indices = [(0, 1), (2, 3), (0, 2), (0, 3), (2, 1), (1, 3)]
fig, axes = pplt.subplots(nrows=3, ncols=6, figwidth=8.5, sharex=False, sharey=False, space=0.3)
for col, (i, j) in enumerate(indices):
    _Z_true = rec.project(Z_true2, [i, j])
    _Z = rec.project(Z, [i, j])
    axes[0, col].pcolormesh(centers[i], centers[j], _Z_true.T, **plot_kws)
    axes[1, col].pcolormesh(centers[i], centers[j], _Z.T, **plot_kws)
    axes[2, col].pcolormesh(centers[i], centers[j], (_Z - _Z_true).T)
    axes[0, col].annotate('{}-{}'.format(labels[i], labels[j]),
                          xy=(0.02, 0.92), xycoords='axes fraction', color='white', 
                          fontsize='medium')
for ax, title in zip(axes[:, 0], ['True', 'Reconstructed', 'Error']):
    ax.set_ylabel(title)
axes.format(xticks=[], yticks=[])
plt.savefig('_output/tomo_sim_rec_art_proj_2D_hor.png', dpi=500)
plt.show()

In [ ]:
plot_kws = dict(cmap='mono_r', robust=False)
indices = [(0, 1), (2, 3), (0, 2), (0, 3), (2, 1), (1, 3)]
fig, axes = pplt.subplots(nrows=6, ncols=3, figsize=(5.0, 8.5), 
                          sharex=False, sharey=False, space=0.5)
for row, (i, j) in enumerate(indices):
    _Z_true = rec.project(Z_true, [i, j])
    _Z = rec.project(Z, [i, j])
    _Z_true2 = rec.project(Z_true2, [i, j])
    axes[row, 0].pcolormesh(_Z_true.T, **plot_kws)
    axes[row, 1].pcolormesh(centers[i], centers[j], _Z.T, **plot_kws)
    axes[row, 2].pcolormesh(centers[i], centers[j], (_Z - _Z_true2).T, 
                            colorbar=True, 
                            colorbar_kw=dict(width=0.075, space=1.0, ticklabelsize='small')
                           )
    axes[row, 0].annotate('{}-{}'.format(labels[i], labels[j]),
                          xy=(0.02, 0.92), xycoords='axes fraction', color='white', 
                          fontsize='medium')
for ax, title in zip(axes[0, :], ['True', 'Reconstructed', 'Error']):
    ax.set_title(title)
axes.format(xticks=[], yticks=[])
plt.savefig('_output/tomo_sim_rec_art_proj2D.png', dpi=350)
plt.show()

## Particle-based 